# LGBM을 활용한 베이스라인

In [1]:
import pandas as pd
import os
import random

## 1. 데이터 로딩

In [4]:
path = '/opt/ml/input/data/'
csv_file_path = os.path.join(path, 'train_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path) 

path = '/opt/ml/input/data/'
dat = pd.read_csv(path + '/FE/FE_total.csv')

In [5]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


In [6]:
dat

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,b_category,test_category,problem_id,category_st_qcut_5
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,6,1,1,"(-0.001, 2.0]"
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3,6,1,2,"(2.0, 12.0]"
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8,6,1,3,"(2.0, 12.0]"
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7,6,1,4,"(2.0, 12.0]"
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7,6,1,5,"(2.0, 12.0]"
...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24,3,71,5,"(12.0, 29.0]"
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0,4,165,1,"(-0.001, 2.0]"
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11,4,165,2,"(2.0, 12.0]"
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46,4,165,3,"(29.0, 66.0]"


In [80]:
# 유저 별 정답률, 푼 문제 개수

tem1 = dat.groupby('userID')['answerCode']
tem1 = pd.DataFrame({'answer_mean' : tem1.mean(), 'answer_cnt':tem1.count()}).reset_index()
tem1

,userID,answer_mean,answer_cnt
0,0,0.630872,745
1,1,0.853162,933
2,2,0.612319,276
3,3,0.691120,1036
4,4,0.691505,671
...,...,...,...
7437,7437,0.375000,16
7438,7438,0.750000,16
7439,7439,0.625000,16
7440,7440,0.400000,15


In [81]:
# 유저별 한 문제 푸는 평균 시간, 총 문제 푼 시간 (초단위)

tem2 = dat.groupby('userID')['solve_time']
tem2 = pd.DataFrame({'time_mean' : tem2.mean(), 'time_sum':tem2.sum()}).reset_index()
tem2

,userID,time_mean,time_sum
0,0,31.628188,23563
1,1,77.018221,71858
2,2,43.478261,12000
3,3,49.230695,51003
4,4,59.967213,40238
...,...,...,...
7437,7437,48.062500,769
7438,7438,49.500000,792
7439,7439,33.187500,531
7440,7440,21.733333,326


In [79]:
# 유저별 가장 많이 푼 태그

tem3 = pd.DataFrame({'tag_mode' : dat.groupby('userID')['KnowledgeTag'].agg(pd.Series.mode)})
tem3

,tag_mode
userID,
0,4784
1,2010
2,"[307, 308]"
3,"[2616, 2622]"
4,2065
...,...
7437,7226
7438,"[1934, 2132]"
7439,2132


In [75]:
# 유저별 최근 3개 문제의 정답률

tem4 = dat.groupby('userID')['answerCode'].apply(lambda x : x.iloc[-3:])
tem4 = pd.DataFrame({'last3_mean' : tem4.groupby('userID').mean()})
tem4

,last3_mean
userID,
0,0.333333
1,1.000000
2,0.000000
3,0.000000
4,0.333333
...,...
7437,0.333333
7438,1.000000
7439,0.333333


In [91]:
user_df = pd.merge(dat, tem1, on=['userID'], how='left')
user_df = pd.merge(user_df, tem2, on=['userID'], how='left')
user_df = pd.merge(user_df, tem3, on=['userID'], how='left')
user_df = pd.merge(user_df, tem4, on=['userID'], how='left')
user_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,b_category,test_category,problem_id,category_st_qcut_5,answer_mean,answer_cnt,time_mean,time_sum,tag_mode,last3_mean
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,6,1,1,"(-0.001, 2.0]",0.630872,745,31.628188,23563,4784,0.333333
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3,6,1,2,"(2.0, 12.0]",0.630872,745,31.628188,23563,4784,0.333333
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8,6,1,3,"(2.0, 12.0]",0.630872,745,31.628188,23563,4784,0.333333
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7,6,1,4,"(2.0, 12.0]",0.630872,745,31.628188,23563,4784,0.333333
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7,6,1,5,"(2.0, 12.0]",0.630872,745,31.628188,23563,4784,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24,3,71,5,"(12.0, 29.0]",0.555556,9,38.888889,350,438,1.000000
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0,4,165,1,"(-0.001, 2.0]",0.555556,9,38.888889,350,438,1.000000
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11,4,165,2,"(2.0, 12.0]",0.555556,9,38.888889,350,438,1.000000
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46,4,165,3,"(29.0, 66.0]",0.555556,9,38.888889,350,438,1.000000


In [104]:
user_df['tag_mode'].astype(int)

ValueError: setting an array element with a sequence.

In [92]:
# 유저별 분리

_train, _test = custom_train_test_split(user_df)

In [97]:
# 사용할 feature 설정

_FEATS = ['answer_mean', 'answer_cnt', 'time_mean', 'time_sum', 'tag_mode', 'last3_mean']

In [93]:
# X, y 값 분리

_train_value = user_df['answerCode']
_train = _train.drop(['answerCode'], axis=1)

_test_value = _test['answerCode']
_test = _test.drop(['answerCode'], axis=1)

In [98]:
lgb_train = lgb.Dataset(_train[_FEATS], _train_value)
lgb_test = lgb.Dataset(_test[_FEATS], _test_value)

In [99]:
_model = lgb.train(
    {'objective': 'binary'}, 
    lgb_train,
    valid_sets=[lgb_train, lgb_test],
    verbose_eval=100,
    num_boost_round=500,
    early_stopping_rounds=100
)

preds = _model.predict(_test[_FEATS])
acc = accuracy_score(_test_value, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(_test_value, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: tag_mode

## 2. Feature Engineering

In [3]:
def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean", 'test_sum']
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_sum']

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    return df

In [4]:
df = feature_engineering(df)
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,test_mean,test_sum,tag_mean,tag_sum
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,NaN,0,NaN,0.947683,1268,0.955022,637
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,1,1.0,0.947683,1268,0.913187,3040
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,2.0,2,1.0,0.947683,1268,0.913187,3040
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,3.0,3,1.0,0.947683,1268,0.913187,3040
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,4.0,4,1.0,0.947683,1268,0.913187,3040


## 3. Train/Test 데이터 셋 분리

In [85]:
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)
def custom_train_test_split(df, ratio=0.7, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [6]:
# 유저별 분리
train, test = custom_train_test_split(df)

# 사용할 Feature 설정
FEATS = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']

# X, y 값 분리
y_train = train['answerCode']
train = train.drop(['answerCode'], axis=1)

y_test = test['answerCode']
test = test.drop(['answerCode'], axis=1)

In [7]:
# !pip install lightgbm

     |████████████████████████████████| 2.0 MB 16.9 MB/s eta 0:00:01


In [94]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

In [95]:
lgb_train = lgb.Dataset(train[FEATS], y_train)
lgb_test = lgb.Dataset(test[FEATS], y_test)

NameError: name 'train' is not defined

## 4. 훈련 및 검증

In [10]:
model = lgb.train(
    {'objective': 'binary'}, 
    lgb_train,
    valid_sets=[lgb_train, lgb_test],
    verbose_eval=100,
    num_boost_round=500,
    early_stopping_rounds=100
)

preds = model.predict(test[FEATS])
acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_test, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1039565, number of negative: 546592
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007573 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2031
[LightGBM] [Info] Number of data points in the train set: 1586157, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.655399 -> initscore=0.642855
[LightGBM] [Info] Start training from score 0.642855
Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.559427	valid_1's binary_logloss: 0.681014
[200]	training's binary_logloss: 0.556528	valid_1's binary_logloss: 0.680677
Early stopping, best iteration is:
[123]	training's binary_logloss: 0.558699	valid_1's binary_logloss: 0.680181
VALID AUC : 0.6904581594116477 ACC : 0.6098654708520179



In [ ]:
# INSTALL MATPLOTLIB IN ADVANCE
# _ = lgb.plot_importance(model)

## 5. Inference

In [11]:
# LOAD TESTDATA
test_csv_file_path = os.path.join(path, 'test_data.csv')
test_df = pd.read_csv(test_csv_file_path)

# FEATURE ENGINEERING
test_df = feature_engineering(test_df)

# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(['answerCode'], axis=1)

In [12]:
# MAKE PREDICTION
total_preds = model.predict(test_df[FEATS])

In [13]:
# SAVE OUTPUT
output_dir = 'output/'
write_path = os.path.join(output_dir, "submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/submission.csv


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

